In [1]:
# Imoports
import pandas as pd
import sqlite3

In [2]:
# Connect SQLite database.
db_conn = sqlite3.connect("SuperstoreDB/superstore.db")

# Questions:

In [3]:
# 1. What is the category generating the maximum sales revenue?
pd.read_sql(
    """
    SELECT Category, SUM(Sales)/100.0 AS "Sales Revenue ($)"
    FROM OrdersDetails
    JOIN Products ON OrdersDetails.ProductID = Products.ProductID
    GROUP BY Category
    ORDER BY "Sales Revenue ($)" DESC
    LIMIT 1;
    """, db_conn)

,Category,Sales Revenue ($)
0,Technology,836149.37


In [4]:
# What about the profit in this category?
pd.read_sql(
    """
    SELECT Category, SUM(Profit)/100.0 AS "Profit ($)"
    FROM OrdersDetails
    JOIN Products ON OrdersDetails.ProductID = Products.ProductID
    GROUP BY Category
    HAVING Category IN (
        SELECT Category
        FROM OrdersDetails
        JOIN Products ON OrdersDetails.ProductID = Products.ProductID
        GROUP BY Category
        ORDER BY SUM(Sales) DESC
        LIMIT 1
    );
    """, db_conn)

,Category,Profit ($)
0,Technology,145448.15


In [5]:
# Are they making a loss in any categories?
# Show the list, so we can see.
pd.read_sql(
    """
    SELECT Category, SUM(Profit)/100.0 AS "Profit ($)"
    FROM OrdersDetails
    JOIN Products ON OrdersDetails.ProductID = Products.ProductID
    GROUP BY Category
    ORDER BY "Profit ($)" DESC;
    """, db_conn)

,Category,Profit ($)
0,Technology,145448.15
1,Office Supplies,122472.33
2,Furniture,18448.03


In [6]:
# Are they making a loss in any categories?
# Displays only categories with negative Profit.
pd.read_sql(
    """
    SELECT Category, SUM(Profit)/100.0 AS "Profit ($)"
    FROM OrdersDetails
    JOIN Products ON OrdersDetails.ProductID = Products.ProductID
    GROUP BY Category
    HAVING "Profit ($)" < 0
    """, db_conn)

,Category,Profit ($)


In [7]:
# No, since there is no negative Profit anywhere.

In [8]:
# Is it also true for sub-categories?
# Show the list, so we can see.
pd.read_sql(
    """
    SELECT SubCategory, SUM(Profit)/100.0 AS "Profit ($)"
    FROM OrdersDetails
    JOIN Products ON OrdersDetails.ProductID = Products.ProductID
    GROUP BY SubCategory
    ORDER BY SUM(Profit);
    """, db_conn)

,SubCategory,Profit ($)
0,Tables,-17725.10
1,Bookcases,-3472.64
2,Supplies,-1189.74
3,Fasteners,948.67
4,Machines,3384.66
5,Labels,5544.73
6,Art,6523.99
7,Envelopes,6963.07
8,Furnishings,13056.10
9,Appliances,18136.42


In [9]:
# Is it also true for sub-categories?
# Displays only sub-categories with negative Profit.
pd.read_sql(
    """
    SELECT SubCategory, SUM(Profit)/100.0 AS "Profit ($)"
    FROM OrdersDetails
    JOIN Products ON OrdersDetails.ProductID = Products.ProductID
    GROUP BY SubCategory
    HAVING "Profit ($)" < 0
    """, db_conn)

,SubCategory,Profit ($)
0,Bookcases,-3472.64
1,Supplies,-1189.74
2,Tables,-17725.10


In [10]:
# Looks like some sub-categories do bring loses.

In [11]:
# 2. What are 5 states generating the maximum and minimum sales revenue?
states_by_sales = pd.read_sql(
    """
    SELECT State, SUM(Sales)/100.0 AS "Sales ($)"
    FROM OrdersDetails
    JOIN Orders ON OrdersDetails.OrderID = Orders.OrderID
    JOIN Addresses ON Orders.AddressID = Addresses.AddressID
    GROUP BY State
    ORDER BY SUM(Sales) DESC
    """, db_conn)

display(states_by_sales.head(5))
display(states_by_sales.tail(5))

,State,Sales ($)
0,California,457683.90
1,New York,310874.30
2,Texas,170184.17
3,Washington,138640.40
4,Pennsylvania,116509.53


,State,Sales ($)
44,Wyoming,1603.13
45,South Dakota,1315.56
46,Maine,1270.53
47,West Virginia,1209.82
48,North Dakota,919.91


In [12]:
# 3. What are the 3 products in each product segment with the highest sales?
# I assume by product segment, Sub-Category is meant.
pd.read_sql(
    """
    SELECT *
    FROM (
        SELECT *,
        RANK() OVER  (
            PARTITION BY SubCategory
            ORDER BY "SalesSum ($)" DESC
        ) AS SalesRank
        FROM
        (
            SELECT Products.ProductID, ProductName, SUM(Sales)/100.0 AS "SalesSum ($)", SubCategory
            FROM OrdersDetails
            JOIN Products ON OrdersDetails.ProductID = Products.ProductID
            GROUP BY Products.ProductID
        )
    )
    WHERE SalesRank <= 3
    """, db_conn)



,ProductID,ProductName,SalesSum ($),SubCategory,SalesRank
0,TEC-AC-10003033,Plantronics CS510 - Over-the-Head monaural Wir...,10822.34,Accessories,1
1,TEC-AC-10004995,Logitech P710e Mobile Speakerphone,10196.59,Accessories,2
2,TEC-AC-10002049,Plantronics Savi W720 Multi-Device Wireless He...,9367.28,Accessories,3
3,OFF-AP-10002945,Honeywell Enviracaire Portable HEPA Air Cleane...,11304.42,Appliances,1
4,OFF-AP-10002651,Hoover Upright Vacuum With Dirt Cup,6832.88,Appliances,2
5,OFF-AP-10000275,Sanyo Counter Height Refrigerator with Crisper...,5906.52,Appliances,3
6,OFF-AR-10000380,"Hunt PowerHouse Electric Pencil Sharpener, Blue",1617.90,Art,1
7,OFF-AR-10002135,Boston Heavy-Duty Trimline Electric Pencil Sha...,1166.44,Art,2
8,OFF-AR-10002671,Hunt BOSTON Model 1606 High-Volume Electric Pe...,1113.02,Art,3
9,OFF-BI-10003527,Fellowes PB500 Electric Punch Plastic Comb Bin...,27453.36,Binders,1


In [13]:
# Are they the 3 most profitable products as well?

In [14]:
# Check the 3 most profitable products for each SubCategory.
pd.read_sql(
    """
    SELECT *
    FROM (
        SELECT *,
        RANK() OVER  (
            PARTITION BY SubCategory
            ORDER BY "ProfitSum ($)" DESC
        ) AS ProfitRank
        FROM
        (
            SELECT Products.ProductID, ProductName, SUM(Profit)/100.0 AS "ProfitSum ($)", SubCategory
            FROM OrdersDetails
            JOIN Products ON OrdersDetails.ProductID = Products.ProductID
            GROUP BY Products.ProductID
        )
    )
    WHERE ProfitRank <= 3
    """, db_conn)

,ProductID,ProductName,ProfitSum ($),SubCategory,ProfitRank
0,TEC-AC-10002049,Plantronics Savi W720 Multi-Device Wireless He...,3696.25,Accessories,1
1,TEC-AC-10003033,Plantronics CS510 - Over-the-Head monaural Wir...,3084.98,Accessories,2
2,TEC-AC-10003870,Logitech Z-906 Speaker sys - home theater - 5....,2243.92,Accessories,3
3,OFF-AP-10002945,Honeywell Enviracaire Portable HEPA Air Cleane...,3246.98,Appliances,1
4,OFF-AP-10000275,Sanyo Counter Height Refrigerator with Crisper...,1653.81,Appliances,2
5,OFF-AP-10003057,Honeywell Enviracaire Portable HEPA Air Cleane...,1503.51,Appliances,3
6,OFF-AR-10000380,"Hunt PowerHouse Electric Pencil Sharpener, Blue",448.11,Art,1
7,OFF-AR-10002135,Boston Heavy-Duty Trimline Electric Pencil Sha...,310.87,Art,2
8,OFF-AR-10001953,Boston 1645 Deluxe Heavier-Duty Electric Penci...,206.23,Art,3
9,OFF-BI-10003527,Fellowes PB500 Electric Punch Plastic Comb Bin...,7753.01,Binders,1


In [15]:
# Compare the tables side to side.
pd.read_sql(
    """
    SELECT * FROM (
        SELECT *
        FROM (
            SELECT *,
            RANK() OVER  (
                PARTITION BY SubCategory
                ORDER BY "SalesSum ($)" DESC
            ) AS SalesRank
            FROM
            (
                SELECT Products.ProductID, ProductName, SUM(Sales)/100.0 AS "SalesSum ($)", SubCategory
                FROM OrdersDetails
                JOIN Products ON OrdersDetails.ProductID = Products.ProductID
                GROUP BY Products.ProductID
            )
        )
        WHERE SalesRank <= 3) AS SalesTable
    JOIN (
        SELECT *
        FROM (
            SELECT *,
            RANK() OVER  (
                PARTITION BY SubCategory
                ORDER BY "ProfitSum ($)" DESC
            ) AS ProfitRank
            FROM
            (
                SELECT Products.ProductID, ProductName, SUM(Profit)/100.0 AS "ProfitSum ($)", SubCategory
                FROM OrdersDetails
                JOIN Products ON OrdersDetails.ProductID = Products.ProductID
                GROUP BY Products.ProductID
            )
        )
        WHERE ProfitRank <= 3) AS ProfitTable
    ON (SalesTable.SubCategory = ProfitTable.SubCategory) AND (SalesTable.SalesRank = ProfitTable.ProfitRank)
    """, db_conn)

,ProductID,ProductName,SalesSum ($),SubCategory,SalesRank,ProductID,ProductName,ProfitSum ($),SubCategory,ProfitRank
0,TEC-AC-10003033,Plantronics CS510 - Over-the-Head monaural Wir...,10822.34,Accessories,1,TEC-AC-10002049,Plantronics Savi W720 Multi-Device Wireless He...,3696.25,Accessories,1
1,TEC-AC-10004995,Logitech P710e Mobile Speakerphone,10196.59,Accessories,2,TEC-AC-10003033,Plantronics CS510 - Over-the-Head monaural Wir...,3084.98,Accessories,2
2,TEC-AC-10002049,Plantronics Savi W720 Multi-Device Wireless He...,9367.28,Accessories,3,TEC-AC-10003870,Logitech Z-906 Speaker sys - home theater - 5....,2243.92,Accessories,3
3,OFF-AP-10002945,Honeywell Enviracaire Portable HEPA Air Cleane...,11304.42,Appliances,1,OFF-AP-10002945,Honeywell Enviracaire Portable HEPA Air Cleane...,3246.98,Appliances,1
4,OFF-AP-10002651,Hoover Upright Vacuum With Dirt Cup,6832.88,Appliances,2,OFF-AP-10000275,Sanyo Counter Height Refrigerator with Crisper...,1653.81,Appliances,2
5,OFF-AP-10000275,Sanyo Counter Height Refrigerator with Crisper...,5906.52,Appliances,3,OFF-AP-10003057,Honeywell Enviracaire Portable HEPA Air Cleane...,1503.51,Appliances,3
6,OFF-AR-10000380,"Hunt PowerHouse Electric Pencil Sharpener, Blue",1617.90,Art,1,OFF-AR-10000380,"Hunt PowerHouse Electric Pencil Sharpener, Blue",448.11,Art,1
7,OFF-AR-10002135,Boston Heavy-Duty Trimline Electric Pencil Sha...,1166.44,Art,2,OFF-AR-10002135,Boston Heavy-Duty Trimline Electric Pencil Sha...,310.87,Art,2
8,OFF-AR-10002671,Hunt BOSTON Model 1606 High-Volume Electric Pe...,1113.02,Art,3,OFF-AR-10001953,Boston 1645 Deluxe Heavier-Duty Electric Penci...,206.23,Art,3
9,OFF-BI-10003527,Fellowes PB500 Electric Punch Plastic Comb Bin...,27453.36,Binders,1,OFF-BI-10003527,Fellowes PB500 Electric Punch Plastic Comb Bin...,7753.01,Binders,1


In [16]:
# Count how many products overlap for each SubCategory?
pd.read_sql(
    """
    WITH SalesTable AS (
        SELECT *
        FROM (
            SELECT *,
                RANK() OVER (
                    PARTITION BY SubCategory
                    ORDER BY "SalesSum ($)" DESC
                ) AS SalesRank
            FROM (
                SELECT Products.ProductID, ProductName, SUM(Sales)/100.0 AS "SalesSum ($)", SubCategory
                FROM OrdersDetails
                JOIN Products ON OrdersDetails.ProductID = Products.ProductID
                GROUP BY Products.ProductID
            )
        )
        WHERE SalesRank <= 3
    ),
    ProfitTable AS (
        SELECT *
        FROM (
            SELECT *,
                RANK() OVER (
                    PARTITION BY SubCategory
                    ORDER BY "ProfitSum ($)" DESC
                ) AS ProfitRank
            FROM (
                SELECT Products.ProductID, ProductName, SUM(Profit)/100.0 AS "ProfitSum ($)", SubCategory
                FROM OrdersDetails
                JOIN Products ON OrdersDetails.ProductID = Products.ProductID
                GROUP BY Products.ProductID
            )
        )
        WHERE ProfitRank <= 3
    )

    -- Count the overlapping ProductIDs grouped by SubCategory
    SELECT ST.SubCategory,
        COUNT(ST.ProductID) AS OverlappingProductIDs
    FROM SalesTable ST
    INNER JOIN ProfitTable PT ON ST.ProductID = PT.ProductID
    GROUP BY ST.SubCategory;
    """, db_conn)

,SubCategory,OverlappingProductIDs
0,Accessories,2
1,Appliances,2
2,Art,2
3,Binders,1
4,Bookcases,1
5,Copiers,3
6,Envelopes,2
7,Fasteners,2
8,Furnishings,1
9,Labels,3


In [17]:
# The products with the highest sales are mostly overlapping quite well with the products that bring the most profit.

In [18]:
# 4. What are the 3 best-seller products in each product segment? (Quantity-wise)
# Again, I assume by product segment, they mean SubCategory
# I use ROW_NUMBER() instead of RANK() or DENSE_Rank() to have only 3 items per SubCategory in the output in case of ties.
pd.read_sql(
    """
    SELECT ProductID, ProductName, QuantitySum, SubCategory
    FROM (
        SELECT *,
        ROW_NUMBER() OVER  (
            PARTITION BY SubCategory
            ORDER BY QuantitySum DESC
        ) AS row_num
        FROM
        (
            SELECT Products.ProductID, ProductName, SUM(Quantity) AS QuantitySum, SubCategory
            FROM OrdersDetails
            JOIN Products ON OrdersDetails.ProductID = Products.ProductID
            GROUP BY Products.ProductID
        )
    )
    WHERE row_num <= 3
    """, db_conn)

,ProductID,ProductName,QuantitySum,SubCategory
0,TEC-AC-10003038,Kingston Digital DataTraveler 16GB USB 2.0,57,Accessories
1,TEC-AC-10003628,Logitech 910-002974 M325 Wireless Mouse for We...,52,Accessories
2,TEC-AC-10004510,Logitech Desktop MK120 Mouse and keyboard Combo,50,Accessories
3,OFF-AP-10000358,Fellowes Basic Home/Office Series Surge Protec...,41,Appliances
4,OFF-AP-10001492,"Acco Six-Outlet Power Strip, 4' Cord Length",41,Appliances
5,OFF-AP-10001205,Belkin 5 Outlet SurgeMaster Power Centers,40,Appliances
6,OFF-AR-10004078,Newell 312,49,Art
7,OFF-AR-10000380,"Hunt PowerHouse Electric Pencil Sharpener, Blue",44,Art
8,OFF-AR-10003514,4009 Highlighters by Sanford,44,Art
9,OFF-BI-10001524,GBC Premium Transparent Covers with Diagonal L...,67,Binders


In [19]:
# 5. What are the top 3 worst-selling products in every category? (Quantity-wise)
pd.read_sql(
    """
    SELECT ProductID, ProductName, QuantitySum, Category
    FROM (
        SELECT *,
        ROW_NUMBER() OVER  (
            PARTITION BY Category
            ORDER BY QuantitySum
        ) AS row_num
        FROM
        (
            SELECT Products.ProductID, ProductName, SUM(Quantity) AS QuantitySum, Category
            FROM OrdersDetails
            JOIN Products ON OrdersDetails.ProductID = Products.ProductID
            GROUP BY Products.ProductID
        )
    )
    WHERE row_num <= 3
    """, db_conn)

,ProductID,ProductName,QuantitySum,Category
0,FUR-BO-10002206,"Bush Saratoga Collection 5-Shelf Bookcase, Han...",1,Furniture
1,FUR-CH-10002317,Global Enterprise Series Seating Low-Back Swiv...,1,Furniture
2,FUR-FU-10002874,Ultra Commercial Grade Dual Valve Door Closer,2,Furniture
3,OFF-AR-10002704,Boston 1900 Electric Pencil Sharpener,1,Office Supplies
4,OFF-PA-10000048,Xerox 20,1,Office Supplies
5,OFF-AP-10002203,Eureka Disposable Bags for Sanitaire Vibra Gro...,2,Office Supplies
6,TEC-MA-10003493,Penpower WorldCard Pro Card Scanner,1,Technology
7,TEC-AC-10003133,Memorex Mini Travel Drive 4 GB USB 2.0 Flash D...,2,Technology
8,TEC-MA-10001856,Okidata C610n Printer,2,Technology


In [20]:
# 6. How many unique customers per month are there for the year 2016.
# (There's a catch here: contrary to other 'heavier' RDBMS, SQLite does not support the functions YEAR() or MONTH() to extract the year or the month in a date.
# You will have to create two new columns: year and month.)
pd.read_sql(
    """
    SELECT Month, COUNT (DISTINCT CustomerID) AS CustomerCount
    FROM (
        SELECT
            *,
            SUBSTR(OrderDate, 1, 4) AS Year,
            SUBSTR(OrderDate, 6, 2) AS Month
        FROM Orders
    )
    WHERE Year = '2016'
    GROUP BY Month

    """, db_conn)

,Month,CustomerCount
0,01,46
1,02,42
2,03,80
3,04,83
4,05,96
5,06,90
6,07,89
7,08,86
8,09,176
9,10,95


In [21]:
# Close the db connection.
db_conn.close()